In [100]:
from __future__ import division
import plotly 
plotly.tools.set_credentials_file(username='Charlie92', api_key='Yy4FPYIxkWvMk4snmW7j')
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)
plotly.tools.set_config_file(world_readable=True,sharing='public')
import plotly.plotly as py
from plotly import graph_objs as go
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

In [101]:
ussd_data = pd.read_excel('ussd-sessions-data.xlsx', sheet_name='ad_hoc-query_12-2e9d2daa1039-20')
 
print("Column headings:")
print(ussd_data.columns)

Column headings:
Index(['id', 'phone_number', 'sessionid', 'newrequest', 'input', 'slug',
       'options', 'created_at'],
      dtype='object')


In [102]:
ussd_data.head()

,id,phone_number,sessionid,newrequest,input,slug,options,created_at
0,1,251953337471,153173803044242,1,911,initial-request,"[{""exists"":true,""id"":""1"",""name"":""Kotegisi"",""ex...",2018-07-16 10:47:11
1,2,251953337471,153173803044242,0,Kigali,ussd-address,NaN,2018-07-16 10:47:15
2,3,251953337471,1531738051111374.back,1,911,initial-request,"[{""exists"":true,""id"":""69"",""name"":""Gahanga"",""ex...",2018-07-16 10:47:32
3,4,251953337471,1531738051111374.back,1,NaN,ussd-address,NaN,2018-07-16 10:47:32
4,5,251953337471,1531738051111374.back,0,Rusororo,ussd-address,NaN,2018-07-16 10:47:40


In [103]:
ussd_data.dtypes

id                       int64
phone_number             int64
sessionid               object
newrequest               int64
input                   object
slug                    object
options                 object
created_at      datetime64[ns]
dtype: object

In [104]:
ussd_data['options'].describe()

count                                                  1295
unique                                                   99
top       [{"exists":true,"id":"50","name":"Kigali","ext...
freq                                                    197
Name: options, dtype: object

In [105]:
ussd_data['slug'].describe()

count                2000
unique                 10
top       initial-request
freq                  647
Name: slug, dtype: object

In [106]:
phases=ussd_data['slug'].unique().tolist()
values=ussd_data['slug'].value_counts().tolist()
print(phases)
print(values)

['initial-request', 'ussd-address', 'selected-category', 'selected-product', 'selected-subscription', 'entered-quantity', 'entered-shipping-method', 'entered-coupon', 'on-demand-payment', 'order-confirmed']
[647, 615, 238, 135, 95, 94, 93, 47, 34, 2]


In [107]:
colors = ['rgb(32,155,160)', 'rgb(253,93,124)', 'rgb(28,119,139)', 'rgb(182,231,235)', 'rgb(35,154,160)','rgb(32,155,160)', 'rgb(253,93,124)', 'rgb(28,119,139)', 'rgb(182,231,235)', 'rgb(35,154,160)']

In [108]:
n_phase = len(phases)
plot_width = 400

# height of a section and difference between sections 
section_h = 100
section_d = 10

# multiplication factor to calculate the width of other sections
unit_width = plot_width / max(values)

# width of each funnel section relative to the plot width
phase_w = [int(value * unit_width) for value in values]

# plot height based on the number of sections and the gap in between them
height = section_h * n_phase + section_d * (n_phase - 1)

In [109]:
# list containing all the plot shapes
shapes = []

# list containing the Y-axis location for each section's name and value text
label_y = []

for i in range(n_phase):
        if (i == n_phase-1):
                points = [phase_w[i] / 2, height, phase_w[i] / 2, height - section_h]
        else:
                points = [phase_w[i] / 2, height, phase_w[i+1] / 2, height - section_h]

        path = 'M {0} {1} L {2} {3} L -{2} {3} L -{0} {1} Z'.format(*points)

        shape = {
                'type': 'path',
                'path': path,
                'fillcolor': colors[i],
                'line': {
                    'width': 1,
                    'color': colors[i]
                }
        }
        shapes.append(shape)
        
        # Y-axis location for this section's details (text)
        label_y.append(height - (section_h / 2))

        height = height - (section_h + section_d)

In [110]:
# For phase names
label_trace = go.Scatter(
    x=[-350]*n_phase,
    y=label_y,
    mode='text',
    text=phases,
    textfont=dict(
        color='rgb(200,200,200)',
        size=15  
    )
)
 
# For phase values
value_trace = go.Scatter(
    x=[350]*n_phase,
    y=label_y,
    mode='text',
    text=values,
    textfont=dict(
        color='rgb(200,200,200)',
        size=15
    )
)

data = [label_trace, value_trace]
 
layout = go.Layout(
    title="<b>CONVERSION FUNNEL</b>",
    titlefont=dict(
        size=20,
        color='rgb(203,203,203)'
    ),
    shapes=shapes,
    height=560,
    width=1000,
    showlegend=False,
    paper_bgcolor='rgba(44,58,71,1)',
    plot_bgcolor='rgba(44,58,71,1)',
    xaxis=dict(
        showticklabels=False,
        zeroline=False,
    ),
    yaxis=dict(
        showticklabels=False,
        zeroline=False
    )
)
 


In [111]:
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)